In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, chdir
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
from datetime import date, timedelta

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

import numpy as np
from epiweeks import Week, Year

In [ ]:
from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.systems import CasosConfirmados
from bulletin.utils.static import Municipios

In [ ]:
cc = CasosConfirmados()
cc.load(f"cc_{hoje.strftime('%d_%m_%Y')}", compress=True)

In [ ]:
cc.df.loc[cc.df['idade']<=11,['uf_residencia','ibge_residencia','idade','sexo','data_diagnostico','evolucao','data_cura_obito']].replace(cc.tables['evolucao'].set_index('id').to_dict()).join(cc.tables['municipios'].set_index('ibge')[['municipio','rs','regional']], on='ibge_residencia').to_excel('casos_obitos_criancas.xlsx', index=False)

In [ ]:
cc.df.loc[cc.df['uf_residencia']!='PR',['uf_residencia','ibge_residencia']] = 'IG',999999

In [ ]:
for col in cc.tables['municipios'].columns:
    if col in cc.df.columns:
        print(col)
        del cc.df[col]

In [ ]:
municipios = cc.tables['municipios'][['uf','macro','rs','regional','ibge','municipio','populacao']]

#populacao
municipiosPR = municipios.loc[municipios['uf'].isin(['PR'])].sort_values(['rs','ibge']).set_index(['uf','macro','rs','regional','ibge','municipio'])
rsPR = municipiosPR.groupby(['rs','regional']).sum()
macroPR = municipiosPR.groupby(['macro']).sum()
PR = municipiosPR.groupby(['uf']).sum()

In [ ]:
dias = pd.date_range(start=date(2020,3,1), end=hoje)
ano = dias.year
mes = dias.month
dia = dias.day
dias = list(zip(ano,mes,dia))
pd.DataFrame(columns=pd.MultiIndex.from_tuples(dias))

In [ ]:
cc.df = cc.df.rename(columns={'ibge_residencia':'ibge'}).join(cc.tables['municipios'].set_index('ibge'), on='ibge')

In [ ]:
casos = cc.df.loc[cc.df['uf']=='PR'].copy()
obitos = casos.loc[casos['evolucao']==2].copy()

In [ ]:
casos['ano'] = casos['data_diagnostico'].dt.year
casos['mes'] = casos['data_diagnostico'].dt.month
casos['dia'] = casos['data_diagnostico'].dt.day

In [ ]:
obitos['ano'] = obitos['data_cura_obito'].dt.year
obitos['mes'] = obitos['data_cura_obito'].dt.month
obitos['dia'] = obitos['data_cura_obito'].dt.day

In [ ]:
casos_mun = casos.pivot_table(index=municipiosPR.index.names, columns=['ano','mes','dia'], values='id_notifica', aggfunc='count', fill_value=0).sort_index(axis=1,level=['ano','mes','dia']).sort_index(axis=0,level=['rs','ibge'])
casos_mun = casos_mun.reindex(municipiosPR.index,axis=0)
casos_mun = casos_mun.reindex(dias,axis=1).fillna(0).astype(int)

casos_rs = casos_mun.pivot_table(index=['rs','regional'],aggfunc='sum')
casos_macro = casos_mun.pivot_table(index='macro',aggfunc='sum')
casos_pr = casos_mun.pivot_table(index='uf',aggfunc='sum')

In [ ]:
obitos_mun = obitos.pivot_table(index=municipiosPR.index.names, columns=['ano','mes','dia'], values='id_notifica', aggfunc='count', fill_value=0).sort_index(axis=1,level=['ano','mes','dia']).sort_index(axis=0,level=['rs','ibge'])
obitos_mun = obitos_mun.reindex(municipiosPR.index,axis=0)
obitos_mun = obitos_mun.reindex(dias,axis=1).fillna(0).astype(int)

obitos_rs = obitos_mun.pivot_table(index=['rs','regional'],aggfunc='sum')
obitos_macro = obitos_mun.pivot_table(index='macro',aggfunc='sum')
obitos_pr = obitos_mun.pivot_table(index='uf',aggfunc='sum')

In [ ]:
def incidencia(casos,populacao,fator=100000):
    assert all(casos.index == populacao.index)
    casos_inc = (casos / populacao.values) * fator
    return casos_inc
    
casos_mun_inc = incidencia(casos_mun,municipiosPR)
casos_rs_inc = incidencia(casos_rs,rsPR)
casos_macro_inc = incidencia(casos_macro,macroPR)
casos_pr_inc = incidencia(casos_pr,PR)

In [ ]:
def mortalidade(obitos,populacao,fator=100000):
    assert all(obitos.index.isin(populacao.index))
    obitos_mort = (obitos / populacao.values) * fator
    return obitos_mort

obitos_mun_mort = mortalidade(obitos_mun,municipiosPR)
obitos_rs_mort = mortalidade(obitos_rs,rsPR)
obitos_macro_mort = mortalidade(obitos_macro,macroPR)
obitos_pr_mort = mortalidade(obitos_pr,PR)

In [ ]:
def letalidade(casos,obitos):
    assert all(casos.index.isin(obitos.index))
    letal = (obitos / casos)
    return letal.replace([np.inf, -np.inf], np.nan)

letal_mun = letalidade(casos_mun,obitos_mun)
letal_rs = letalidade(casos_rs,obitos_rs)
letal_macro = letalidade(casos_macro,obitos_macro)
letal_pr = letalidade(casos_pr,obitos_pr)

In [ ]:
# a = incidencia(casos_se.loc[:,(2022)].sum(axis=1).to_frame(name='janeiro')).groupby(['rs','ibge','municipio']).sum()
# a['rank'] = a.rank(ascending=False)
# a.sort_values('rank').head(10)
# a

In [ ]:
writer = pd.ExcelWriter(join(default_output,"coef.xlsx"))

In [ ]:
casos_pr.to_excel(writer,'casos_pr')
casos_pr_inc.to_excel(writer,'casos_pr_inc')
obitos_pr.to_excel(writer,'obitos_pr')
obitos_pr_mort.to_excel(writer,'obitos_pr_mort')
letal_pr.to_excel(writer,'letal_pr')

In [ ]:
casos_macro.to_excel(writer,'casos_macro')
casos_macro_inc.to_excel(writer,'casos_macro_inc')
obitos_macro.to_excel(writer,'obitos_macro')
obitos_macro_mort.to_excel(writer,'obitos_macro_mort')
letal_macro.to_excel(writer,'letal_macro')

In [ ]:
casos_rs.to_excel(writer,'casos_rs')
casos_rs_inc.to_excel(writer,'casos_rs_inc')
obitos_rs.to_excel(writer,'obitos_rs')
obitos_rs_mort.to_excel(writer,'obitos_rs_mort')
letal_rs.to_excel(writer,'letal_rs')

In [ ]:
casos_mun.to_excel(writer,'casos_mun')
casos_mun_inc.to_excel(writer,'casos_mun_inc')
obitos_mun.to_excel(writer,'obitos_mun')
obitos_mun_mort.to_excel(writer,'obitos_mun_mort')
letal_mun.to_excel(writer,'letal_mun')

In [ ]:
writer.save()

In [ ]:
casos_mun

In [ ]:
casos_pr_inc